In [ ]:
import math
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
from tabulate import tabulate

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

# The following imports NEED to be in the exact order
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from validation.tokeneconomy.baseline import economyconfig
from validation.tokeneconomy.baseline.economyconfig import experiment

from validation.plot_functions import *

exec_mode = ExecutionMode()

In [ ]:
configs = experiment.configs

In [ ]:
single_proc_ctx = ExecutionContext(context=exec_mode.single_proc)
run1 = Executor(exec_context=single_proc_ctx, configs=[configs[0]])
run1_raw_result, raw_tensor_field = run1.execute()
raw_df = pd.DataFrame(run1_raw_result)
df = raw_df[raw_df['substep']==5]

In [ ]:
pd.options.mode.chained_assignment = None
df['lambda_S_factor'] = df['R'] / df['R_prev']
df['lambda_D_factor'] = df['P_prev'] / df['P']

# factors going into V
# V(t) ~ (P(t) * Q(t) + B(t)) / (C(t) * Q(t))
df['CQ'] = df['C'] * df['Q']
df['PQ_K'] = df['P'] * df['Q']
df['PQplusB_K'] = df['P'] * df['Q'] + df['B']
df['PQ_FIAT'] = df['P'] * df['Q'] * df['K']
df['PQplusB_FIAT'] = (df['P'] * df['Q'] + df['B']) * df['K']
df['PoverC'] = df['P'] / df['C']
df['BoverCQ'] = df['B'] / df['CQ']
df['P_FIAT'] = df['P'] * df['K']
df['V_inverse'] = 1.0 / df['V']
df['log_K'] = np.log(df['K'])
df['log_P_FIAT'] = np.log(df['P_FIAT'])
# df['delta_KPI'] = df['KPI'] - df['KPI_prev']
df['PprevOverP'] = df['P_prev'] / df['P']
df['QoverQprev'] = df['Q'] / df['Q_prev']
df['QoverQ0'] = df['Q'] / df['Q'].iloc[0]

# supply - demand
df['SdiffD'] = df['S'] - df['D']
df['DdiffS'] = df['D'] - df['S']

In [ ]:
aggregate_dimension = 'timestep'

mean_df = df.groupby(aggregate_dimension).mean().reset_index()
median_df = df.groupby(aggregate_dimension).median().reset_index()
std_df = df.groupby(aggregate_dimension).std().reset_index()
min_df = df.groupby(aggregate_dimension).min().reset_index()
df.rename(columns={'weeks': "timestep"}, inplace=True)

### Simulation - simple exponential decay block rewards

In [ ]:
dist_plot(df, 'timestep', 'B')

#### K - secondary market price of token

In [ ]:
plt.figure()
dist_plot(df, 'timestep', 'log_K', suppMin=True)
title_text = 'Monte Carlo distribution over token prices (log scale)'
plt.title(title_text)
plt.show()

In [ ]:
first_five_plot(df, 'log_K')

#### Q - service transacted on the platform

In [ ]:
terminal_Q = df.groupby('run').last().reset_index()['Q']
initial_Q = df.groupby('run').first().reset_index()['Q']
normalized_Q = terminal_Q / initial_Q
plt.figure(figsize=(10, 6))
normalized_Q.hist(normed=True)
sns.kdeplot(normalized_Q)
plt.xlabel('Aggregated Growth in Q')
title_text = 'Monte Carlo distribution over aggregated growth in Q'
plt.title(title_text)
plt.show()